In [ ]:

**************In Cypher, labels, property keys, and variables are case-sensitive. Cypher keywords are not case-sensitive.**********

(m:Movie {title: 'Cloud Atlas'})<-[:ACTED_IN]-(p:Person)

MATCH (p:Person)
RETURN p

MATCH (p:Person {name: 'Tom Hanks'})
RETURN p


MATCH (p:Person {name: 'Tom Hanks'})
RETURN  p.born


MATCH (p:Person)
WHERE p.name = 'Tom Hanks'
RETURN p.born

MATCH (p:Person)
WHERE p.name = 'Tom Hanks' OR p.name = 'Rita Wilson'
RETURN p.name, p.born


***************************Relationship*********************************************************
MATCH (p:Person {name: 'Tom Hanks'})
RETURN p

MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m)
RETURN m.title

or

MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie)
RETURN m.title


MATCH (m:Movie {title: 'Cloud Atlas'})<-[:DIRECTED]-(p:Person)  RETURN COUNT(p)

MATCH (:Person {name: 'Emil Eifrem'})-[:ACTED_IN]->(m:Movie)
RETURN  m.title

***************************************Filtering Queries*********************************************
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.released = 2008 OR m.released = 2009
RETURN p, m

MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title='The Matrix'
RETURN p.name


OR

MATCH (p)-[:ACTED_IN]->(m)
WHERE p:Person AND m:Movie AND m.title='The Matrix'
RETURN p.name



MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE 2000 <= m.released <= 2003
RETURN p.name, m.title, m.released



MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name='Jack Nicholson' AND m.tagline IS NOT NULL
RETURN m.title, m.tagline



MATCH (p:Person)-[:ACTED_IN]->()
WHERE p.name STARTS WITH 'Michael'
RETURN p.name



MATCH (p:Person)-[:ACTED_IN]->()
WHERE toLower(p.name) STARTS WITH 'michael'
RETURN p.name



MATCH (p:Person)-[:ACTED_IN]->()
WHERE toUpper(p.name) STARTS WITH 'michael'
RETURN p.name



Suppose you wanted to find all people who wrote a movie but did not direct that same movie.
Here is how you would perform the query.

MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE NOT exists( (p)-[:DIRECTED]->(m) )
RETURN p.name, m.title


we only want to retrieve Person nodes of people born in 1965, 1970, or 1975:



MATCH (p:Person)
WHERE p.born IN [1965, 1970, 1975]
RETURN p.name, p.born



MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE  'Neo' IN r.roles AND m.title='The Matrix'
RETURN p.name, r.roles




MATCH (p:Person)
RETURN p.name, keys(p)




To return all the property keys defined in the graph.

CALL db.propertyKeys()



MATCH (a:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.title='The Matrix' AND a.born > 1960
RETURN count(a)

**************************Writing Data****************************************
1. Create Node.

MERGE (p:Person {name: 'Michael Caine'})


MATCH (p:Person {name: 'Michael Caine'})
RETURN p


2.We can also chain multiple MERGE clauses together within a single Cypher code block.

MERGE (p:Person {name: 'Katie Holmes'})
MERGE (m:Movie {title: 'The Dark Knight'})
RETURN p, m


*******************************************************************************
Using CREATE instead of MERGE to create nodes
Cypher has a CREATE clause you can use for creating nodes.
The benefit of using CREATE is that it does not look up the primary key before adding the node.
You can use CREATE if you are sure your data is clean and you want greater speed during import.
We use MERGE in this training because it eliminates duplication of nodes.
*************************************************************************************



***********************************CREATE RELATIONSHIP BET TWO NODES*******************
MATCH (p:Person {name: 'Michael Caine'})
MATCH (m:Movie {title: 'The Dark Knight'})
MERGE (p)-[:ACTED_IN]->(m)

MATCH (p:Person {name: 'Michael Caine'})-[:ACTED_IN]-(m:Movie {title: 'The Dark Knight'})
RETURN p, m

This query returns no nodes since there are no nodes with the ACTED_IN relationship to Person nodes in the graph.

MATCH (p:Person {name: 'Michael Caine'})<-[:ACTED_IN]-(m:Movie {title: 'The Dark Knight'})
RETURN p, m



MERGE (p:Person {name: 'Chadwick Boseman'})
MERGE (m:Movie {title: 'Black Panther'})
MERGE (p)-[:ACTED_IN]-(m)


Note that in this MERGE clause where we create the relationships,
we did not specify the direction of the relationship.
By default, if you do not specify the direction when you create the relationship,
it will always be assumed left-to-right.


MATCH (p:Person {name: 'Chadwick Boseman'})-[:ACTED_IN]-(m:Movie {title: 'Black Panther'})
RETURN p, m



MERGE (p:Person {name: 'Emily Blunt'})-[:ACTED_IN]->(m:Movie {title: 'A Quiet Place'})
RETURN p, m



1.Find the Person node for Daniel Kaluuya.

Create the Movie node, Get Out.

Add the ACTED_IN relationship between Daniel Kaluuya and the movie, Get Out.

NOTE:
You will do a MATCH to find the person. Then you will do a MERGE to create the movie node.
Remember that our other Movie nodes contain a property called title.
Then you will do a MERGE to create the relationship.

MATCH (p:Person) WHERE p.name = 'Daniel Kaluuya'
MERGE (m:Movie {title: 'Get Out'})
MATCH (p)-[:ACTED_IN]->(m)
RETURN p, m


*****************************Updating Properties************************************************
1.In this code, the actor, Michael Caine exists but the movie,
Batman Begins does not. We find the Person node and we create the Movie node.
Then, we create the ACTED_IN relationship between the Michael Caine node and
the newly-created Batman Begins node. And we set the roles property for this
relationship to an array of values - containing one value, Alfred Penny.
Notice that for inline property setting, we use the JSON-style of adding
the property key/value pairs in braces { .. }, just like we did when
we specified the property for the node.




MATCH (p:Person {name: 'Michael Caine'})
MERGE (m:Movie {title: 'Batman Begins'})
MERGE (p)-[:ACTED_IN {roles: ['Alfred Penny']}]->(m)
RETURN p,m


2.We also have the option to use the SET keyword for setting a property value.
 In the context of particular MERGE or MATCH clause where you have defined
a variable to reference the node or relationship, you can set property values.


MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
SET r.roles = ['Alfred Penny']
RETURN p, r, m


3.If you need to set multiple properties, you separate them with a comma (,).

MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
SET r.roles = ['Alfred Penny'], m.released = 2008
RETURN p, r, m


4.Updating properties.


MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
SET r.roles = ['Mr. Alfred Penny']
RETURN p, r, m


5.Removing properties.


MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
REMOVE r.roles
RETURN p, r, m

6.remove the born property.

MATCH (p:Person)
WHERE p.name = 'Gene Hackman'
SET p.born = null
RETURN p


NOTE:You should never remove the property that is used as the primary key for a node.

MATCH (m:Movie {title: 'Get Out'})
SET m.tagline='Gripping, scary, witty and timely!',m.released= 2017
RETURN m.title, m.tagline, m.released


********************MERGE Processing*************************************************

// Find or create a person with this name
MERGE (p:Person {name: 'McKenna Grace'})

// Only set the `createdAt` property if the node is created during this query
ON CREATE SET p.createdAt = datetime()

// Only set the `updatedAt` property if the node was created previously
ON MATCH SET p.updatedAt = datetime()

// Set the `born` property regardless
SET p.born = 2006

RETURN p

-----------------------------------------------------------------------
NOTE:If you want to set multiple properties for an ON CREATE SET or ON MATCH SET clause,
you separate them by commas.

ON CREATE SET m.released = 2020, m.tagline = `A great ride!'

-------------------------------------------------------------------------
2.Merging Relationship.


// Find or create a person with this name
MERGE (p:Person {name: 'Michael Caine'})

// Find or create a movie with this title
MERGE (m:Movie {title: 'The Cider House Rules'})

// Find or create a relationship between the two nodes
MERGE (p)-[:ACTED_IN]->(m)

OR

MERGE (p:Person {name: 'Michael Caine'})-[:ACTED_IN]->(m:Movie {title: 'The Cider House Rules'})
RETURN p, m


*************************************************************************************


/* First Run - only  createdAt and updatedAt will be set */
MERGE (m:Movie {title: 'Rocketman'})
/* perform the ON MATCH setting of the matchedAt property */
ON MATCH SET m.matchedAt = datetime()
/* perform the ON CREATE setting of the createdAt property */
ON CREATE SET m.createdAt = datetime()
/* set the updatedAt property */
SET m.updatedAt = timestamp()
RETURN m.title, m.createdAt, m.matchedAt, m.updatedAt;


/* Second Run - all three properties will be set */
MERGE (m:Movie {title: 'Rocketman'})
/* perform the ON MATCH setting of the matchedAt property */
ON MATCH SET m.matchedAt = datetime()
/* perform the ON CREATE setting of the createdAt property */
ON CREATE SET m.createdAt = datetime()
/* set the updatedAt property */
SET m.updatedAt = timestamp()
RETURN m.title, m.createdAt, m.matchedAt, m.updatedAt

*************************DELETE*****************************************************************
MERGE (p:Person {name: 'Jane Doe'})


MATCH (p:Person)
WHERE p.name = 'Jane Doe'
DELETE p


MATCH (m:Movie {title: 'The Matrix'})
MERGE (p:Person {name: 'Jane Doe'})
MERGE (p)-[:ACTED_IN]->(m)
RETURN p, m



To leave the Jane Doe node in the graph, but remove the relationship we retrieve the relationship and delete it.


MATCH (p:Person {name: 'Jane Doe'})-[r:ACTED_IN]->(m:Movie {title: 'The Matrix'})
DELETE r
RETURN p, m


Run this Cypher code that recreates the relationship:



MATCH (p:Person {name: 'Jane Doe'})
MATCH (m:Movie {title: 'The Matrix'})
MERGE (p)-[:ACTED_IN]->(m)
RETURN p, m



If we attempt to delete the Jane Doe node, we will receive an error because it has relationships in the graph.

MATCH (p:Person {name: 'Jane Doe'})
DELETE p




Deleting a node and its relationships
Neo4j provides a feature where you cannot delete a node if it has incoming or outgoing relationships.
This prevents the graph from having orphaned relationships.



MATCH (p:Person {name: 'Jane Doe'})
DETACH DELETE p



Do NOT delete all nodes and relationships in the graph as you need them for the next challenge!

MATCH (n)
DETACH DELETE n



NOTE:You should only do this on relatively small databases as trying
to do this on a large database will exhaust memory.


HOW TO DELETE A LABEL?

Run this Cypher code to create this Person node in the graph:



MERGE (p:Person {name: 'Jane Doe'})
RETURN p



Next, run this code to add a new label to this node:


MATCH (p:Person {name: 'Jane Doe'})
SET p:Developer
RETURN p


To remove the newly-added label, Developer, you use the REMOVE clause. Run this code:


MATCH (p:Person {name: 'Jane Doe'})
REMOVE p:Developer
RETURN p


NOTE:The Jane Doe node has two labels, Person and Developer.
You can use a MATCH to find that node.
Note that you could have specified MATCH (p:Developer {name: 'Jane Doe'})
or MATCH (p:Person:Developer {name: 'Jane Doe'}) to find the same node.
Once we have a reference to that node, we can remove the label with the REMOVE clause.


And finally, delete the Jane Doe node by running this code:



MATCH (p:Person {name: 'Jane Doe'})
DETACH DELETE p


This code returns all node labels defined in the graph.



CALL db.labels()
*******************************************************************************************





































